In [20]:
# 알라딘, 위키북스

import requests
import lxml.html
import re
import time

def main():
    session = requests.Session()
    response = session.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BranchType=1")
    urls = page_list(response)
  
    for url in urls:
        response=session.get(url)
        bookINFO=page_detail(response)
        print(bookINFO)
    session.close()

def page_list(response):
    root = lxml.html.fromstring(response.text)
    #root.make_links_absolute(response.url)
    count=0
    for a in root.cssselect('a.bo3'):
        url = a.get('href')
        
        yield url
        count+=1
    print(count)

def normalize_space(s):
    return re.sub(r'\s+',' ',s).strip()


In [21]:
def page_detail(response):
    root = lxml.html.fromstring(response.text)
    bookINFO= {
        'url' : response.url,
        'title': root.cssselect(".pwrap_bgtit a")[0].text_content(),
        'price' : root.cssselect("td.p_goodstd02")[0].text_content()
        #'content' : [normalize_space(p.text_content()) for p in root.cssselect("#tabs_3 .hanbit_edit_view p")
        #                                if normalize_space(p.text_content()) != '']
    }
    return bookINFO

if __name__ == "__main__":
    main()



{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=168915737', 'title': '골든아워 1', 'price': '15,800원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=174495561', 'title': '걷는 사람, 하정우', 'price': '15,500원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=174571448', 'title': '추리 천재 엉덩이 탐정 7', 'price': '12,000원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=172838153', 'title': '2019 수상한 임용 심층면접', 'price': '29,800원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=168915857', 'title': '골든아워 2', 'price': '15,800원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=169387318', 'title': '당신이 옳다', 'price': '15,800원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=170467155', 'title': '트렌드 코리아 2019', 'price': '17,000원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=171686552', 'title': '네이티브 영어표현력 사전', 'price': '25,000원 '}
{'url': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=173993981

In [10]:
'''
step01 단계 확장하기
- 향후 완결 로직 : main page에서 소개하는 모든 책들의 상세 page의 목차 스크래핑
- 권장 구조
    1. 로직별 개별 함수로 개발 권장
    2. 다수의 page를 스크래핑 해야 하는 관계로
        - 연속적인 크롤링. 따라서 세션이라는 객체가 필요
- 사고력 기르기  Session
    1. 연속적인 다수의 page 크롤링시 세션(Session)을 사용하는 사유?
        - 연결 유지
        - 서버의 리소스 자원을 절약할수 있게 하는 매너있는 자세

- 주요 API
    lxml.html : tree 기반의 html 문서를 다룰 수 있게 지원해주는 api

- 개발 방식
    1. process
        세션 객체 생성 -> 크롤링 -> lxml을 사용하여 html에서 데이터 추출
    2. 코드
        request.Session()
        get
        lxml.html.fromstring()

- 용어 정리
1. 세션
    - 로그인~로그아웃 할때까지 어떤 user인지 구분, 관리 - 상태유지
    - 상태 유지 기술
    - 세션 처리를 가장 예민 하게 관리 : 단연코 금융
2. 

'''
import requests
import lxml.html
import re
import time

def main():
    session = requests.Session()
    response = session.get("http://www.hanbit.co.kr/store/books/new_book_list.html")
    urls = scrape_list_page(response)

    for url in urls:
        time.sleep(1)
        response=session.get(url)
        bookINFO=scrape_detail_page(response)
        print(bookINFO)
    session.close()

def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('#container > div.new_book_list_wrap > div.sub_book_list_area > li:nth-child(1) > div > div > p.book_tit > a'):
        url = a.get('href')
        print("**1")
        yield url 


#상세 page에서 목차 정보 스크래핑 하는 함수
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    bookINFO= {
        'url' : response.url,
        'title': root.cssselect(".store_product_info_box > h3")[0].text_content(),
        'price' : root.cssselect(".pbr del")[0].text_content(),
        'content' : [normalize_space(p.text_content()) for p in root.cssselect("#tabs_3 .hanbit_edit_view p")
                                        if normalize_space(p.text_content()) != '']
    }
    return bookINFO

'''
1.문자열 앞뒤에 잉여 여백 제거 : strip()
2. '' 무의미한 문자열 데이터 제거
    : if 조건식으로 필터링
        if(p.text_content()!='')
3. 문자열들 사이에 다수의 의미없는 여백이 존재할 경우 하나의 여백으로 가공 
    : re.sub(수정하고자하는 표현식, 변경할 표현식, 데이터)
      re.sub('\s+', ' ', 데이터)
'''

def normalize_space(s):
    return re.sub(r'\s+',' ',s).strip()

'''
    상세 정보는 dict 구조로 가공하기
    key - url, title, price, content(목차)
    value = cssselect()
                이 함수의 반환타입은 list
'''


if __name__ == "__main__":
    main()




**1
{'url': 'http://www.hanbit.co.kr/store/books/look.php?p_code=B6270769200', 'title': '오토캐드 트레이닝 북', 'price': '29,000원', 'content': ['PART 01 AutoCAD 작업 환경 준비하기', 'LESSON 01 AutoCAD 화면 살펴보기', '시작 화면', '새 도면 파일 만들고 저장하기', '작업 화면의 구성 요소', '[보충 강의] 파일 백업하고 복구하기', 'LESSON 02 효율적인 작업을 위한 기본 설정 변경하기', '옵션 설정하기', '화면 구성 요소의 가시성 조절하기', '상태막대 설정하기', '프로그램 초기화', 'LESSON 03 화면 확대•축소 및 이동하기', '마우스 휠과 바로 가기 메뉴로 화면 제어하기', '[실습 예제] 화면 제어하기', '[보충 강의] 시스템 변수란?', 'LESSON 04 명령 입력하기', '리본 메뉴에서 명령 실행하기', '직접 명령 입력하기', '단축 명령 입력하기', '[실습 예제] LINE•RECTANGLE•CIRCLE 명령의 단축 명령 연습하기', '[보충 강의] 단축 명령 수정하기', 'LESSON 05 객체 선택하기', '객체 선택하고 해제하기', '선택 방향에 따른 윈도우 선택과 걸침 선택', '올가미 선택과 울타리, 윈도우 폴리곤과 걸침 폴리곤', '순환 선택과 OVERKILL 명령', '비슷한 속성을 갖는 객체를 빠르게 선택하는 유사 선택과 신속 선택', 'PART 02 기본 작도 명령과 편집 명령', 'LESSON 01 극좌표 추적, 직교 모드, 객체 스냅으로 선 긋기', 'AutoCAD의 단위 체계', '직선을 그리는 세 가지 방법', '극좌표 추적과 직교 모드', '객체 스냅', '[실습 예제] 직교 모드와 객체 스냅을 활용하여 도면 작성하기', '[BASIC TRAINING] 극좌표 추적과 직교 모드를 활용한 작도 연습', 'LESSON 02 상대좌표로 선 긋기', '절대좌표와 상대좌표',